In [15]:
import os
import PyPDF2
import random

def extract_text_from_pdfs(pdf_folder):
    all_text = ''
    for filename in os.listdir(pdf_folder):
        if filename.endswith('.pdf'):
            path = os.path.join(pdf_folder, filename)
            with open(path, 'rb') as file:
                reader = PyPDF2.PdfReader(file)
                for page in reader.pages:
                    all_text += page.extract_text() + '\n'
    return all_text

pdf_folder = './pdfs'  # altere para o caminho real
corpus_text = extract_text_from_pdfs(pdf_folder)

print(corpus_text)

Mercado Ver-o-Peso
Cultura e turismo  de Belém (Pará)
Cultura e turismo de Belém  são diversificadas as opções culturais e turísticas da capital do
Estado do pará , influenciadas por indígenas e imigrantes estrangeiros, que manifestam-se através
das manifestações religiosas, da gastronomia, do folclore, danças, músicas, teatros, museus, etc. A
capital paraense desponta como grande roteiro turístico do Brasil, gerando uma excelente
oportunidade para investimentos turísticos.
Bioparque Amazônia  (1989 ): parque particular
criado pelo médico Jorge Aarão
Monteiro.[1][2][3][4][5]
Bondinho de Belém: tecnologia implantada na
época da Belle Époque  (1871-1914).
Casa das Onze Janelas  (século XVIII ,
aproximadamente 1750 ): vizinho do Forte do
Castelo, feito pelo dono de engenho de açúcar  Domingos da Costa Bacelar .[6][7][8]
Complexo Feliz Lusitânia  (1616 ): um povoado colonial português  criado pelo capitão Francisco
Caldeira Castelo Branco  às margens da baía do Guajará. Nucleo inicial de B

In [ ]:
import re
import nltk
from nltk.tokenize import sent_tokenize

nltk.download('punkt',download_dir='./nltk_data', quiet=True)

def preprocess_text(text):
    text = text.lower()
    # text = re.sub(r'\n+', ' ', text)
    # # MANTÉM . ! ? para detecção de fim de frase
    # text = re.sub(r'[^a-záéíóúàãõç\.\!\?\s]', '', text)
    # return text
    text = text.replace('\xa0', ' ')
    text = re.sub(r'\n+', '\n', text)  # mantém quebras de parágrafo
    text = re.sub(r'[ \t]+', ' ', text)
    text = re.sub(r' +\n', '\n', text)
    return text.strip()

# corpus_text = "Belém é a capital do Pará. É conhecida pelo Círio de Nazaré! Você já visitou?"
processed_text = preprocess_text(corpus_text)
# clean_text = sent_tokenize(processed_text, language='portuguese')


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/portuguese/[0m

  Searched in:
    - '/home/prinako/nltk_data'
    - '/home/prinako/External/School/UFPA/ITEC_UFPA/24.4_SEMESTER/INTELIGENCIA_COMPUTACIONAL/T2/amazon-portuguese-chatbot/venv/nltk_data'
    - '/home/prinako/External/School/UFPA/ITEC_UFPA/24.4_SEMESTER/INTELIGENCIA_COMPUTACIONAL/T2/amazon-portuguese-chatbot/venv/share/nltk_data'
    - '/home/prinako/External/School/UFPA/ITEC_UFPA/24.4_SEMESTER/INTELIGENCIA_COMPUTACIONAL/T2/amazon-portuguese-chatbot/venv/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
***********************************

In [4]:
def split_paragraphs_into_sentences(text):
    paragraphs = text.split('\n')
    sentences = []
    for paragraph in paragraphs:
        parts = re.split(r'(?<=[.!?])\s+', paragraph.strip())
        for part in parts:
            part = part.strip()
            if len(part) > 30:
                sentences.append(part)
    return sentences

# === 4. Gerar perguntas/respostas ===
def generate_qa_pairs(sentences):
    qa_pairs = []
    for i in range(1, len(sentences)):
        prev = sentences[i-1]
        curr = sentences[i]
        question = f"O que você pode me dizer sobre: '{prev[:60]}...'"
        answer = curr
        qa_pairs.append((question, answer))
    return qa_pairs

def generate_qa_pairs_improved(sentences, min_words=6, max_chars=180):
    question_templates = [
        "Você pode me explicar sobre \"{}\"?",
        "O que é \"{}\"?",
        "Fale sobre \"{}\".",
        "O que você sabe sobre \"{}\"?",
        "Qual a importância de \"{}\"?",
        "Conte-me algo sobre \"{}\"."
    ]

    qa_pairs = []
    for i in range(len(sentences) - 1):
        context = sentences[i].strip()
        response = sentences[i+1].strip()

        # Filtro: só usar como pergunta se tiver palavras suficientes
        if len(context.split()) >= min_words and len(response.split()) >= min_words:
            # Garante que termina com pontuação
            context_clean = re.sub(r'\s+', ' ', context)
            context_clean = context_clean.strip()

            # Se for muito longo, corta no último ponto final, interrogação ou exclamação antes do limite
            if len(context_clean) > max_chars:
                match = re.search(r'^(.{30,' + str(max_chars) + r'}[.!?])', context_clean)
                if match:
                    context_clean = match.group(1)

            question_text = random.choice(question_templates).format(context_clean)
            qa_pairs.append((question_text, response))

    return qa_pairs



In [5]:
sentences = split_paragraphs_into_sentences(processed_text)
qa_pairs = generate_qa_pairs_improved(sentences)
# qa_pairs

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Separar perguntas e respostas
perguntas = [q for q, _ in qa_pairs]
respostas = [a for _, a in qa_pairs]

# Tokenizer
tokenizer = Tokenizer(num_words=5000,oov_token="<OOV>")
tokenizer.fit_on_texts(perguntas + respostas)

word_index = tokenizer.word_index
vocab_size = len(word_index) + 1

# Sequenciar e padronizar
max_len = 40  # pode ajustar
X = tokenizer.texts_to_sequences(perguntas)
y = tokenizer.texts_to_sequences(respostas)

X = pad_sequences(X, maxlen=max_len, padding='post')
y = pad_sequences(y, maxlen=max_len, padding='post')

# Ajustar formato do y para categorical_crossentropy
y = np.expand_dims(y, -1)


2025-03-31 23:55:36.812247: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743476136.824010  154352 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743476136.827448  154352 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1743476136.837337  154352 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743476136.837353  154352 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1743476136.837355  154352 computation_placer.cc:177] computation placer alr

In [7]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense

embedding_dim = 128

input_seq = Input(shape=(max_len,))
embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim)(input_seq)
lstm = LSTM(128, return_sequences=True)(embedding)
output = Dense(vocab_size, activation='softmax')(lstm)

model = Model(input_seq, output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()


I0000 00:00:1743476139.144899  154352 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2167 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 40)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 40, 128)        │       378,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 40, 128)        │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 40, 2956)       │       381,324 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 891,276 (3.40 MB)

 Trainable params: 891,276 (3.40 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
model.fit(X, y, epochs=100, batch_size=16, validation_split=0.1)


Epoch 1/100


I0000 00:00:1743476141.249974  154848 cuda_dnn.cc:529] Loaded cuDNN version 90300


50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.6433 - loss: 5.9163 - val_accuracy: 0.7315 - val_loss: 2.3470
Epoch 2/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7105 - loss: 2.3812 - val_accuracy: 0.7315 - val_loss: 2.2323
Epoch 3/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7133 - loss: 2.1947 - val_accuracy: 0.7315 - val_loss: 2.1811
Epoch 4/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7138 - loss: 2.1059 - val_accuracy: 0.7315 - val_loss: 2.1743
Epoch 5/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7106 - loss: 2.0797 - val_accuracy: 0.7315 - val_loss: 2.1858
Epoch 6/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7169 - loss: 2.0019 - val_accuracy: 0.7346 - val_loss: 2.2124
Epoch 7/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7190 - loss: 1.9748 - val_accuracy: 0.7357 - val_loss: 2.2379
Epoch 8/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7257 - loss: 1.9157 - val_accuracy: 0.7323 - val_loss: 2.

In [14]:
def responder(pergunta, max_len=400):
    seq = tokenizer.texts_to_sequences([pergunta])
    padded = pad_sequences(seq, maxlen=max_len, padding='post')
    pred = model.predict(padded)[0]
    print(pred.shape)
    pred_ids = np.argmax(pred, axis=-1)

    resposta = " ".join([tokenizer.index_word.get(i, '') for i in pred_ids if i != 0])
    return resposta.strip()

# Exemplo:
print(responder("O que é o Ver-o-Peso?"))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
(400, 2956)
o de o de a a do
